<a href="https://colab.research.google.com/github/AlexanderKClark/MAT421/blob/main/MAT421_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MAT421 HW4 - Alexander Clark
This homework covers the concepts in [Chapter 19 of the textbook](https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter19.00-Root-Finding.html): **Root Finding Problem Statement**, **Tolerance**, **Bisection Method**, **Newton-Raphson Method**, and **Root Finding in Python**.

## 19.1 Root Finding Problem Statement
Determining analytic solutions to some functions can be difficult, and it can be beneficial to generate a numerical approximation of a given root so long as the limitations in doing so is understood.

For example, consider the roots of $f(x)=\text{cos}(x)-x$.

In [7]:
import numpy as np
from scipy import optimize

f = lambda x: np.cos(x) - x
r = optimize.fsolve(f, -2)
print("r =", r)

# Verify the solution is a root
result = f(r)
print("result=", result)

r = [0.73908513]
result= [0.]


The function $f(x)=\frac{1}{x}$ has no root, and when *fsolve* tries to compute the root *full_output* is a boolean argument that can be added to *fsolve* to troubleshoot.

In [8]:
f = lambda x: 1/x

r, infodict, ier, mesg = optimize.fsolve(f, -2, full_output=True)
print("r =", r)

result = f(r)
print("result=", result)

print(mesg)

r = [-3.52047359e+83]
result= [-2.84052692e-84]
The number of calls to function has reached maxfev = 400.


## 19.2 Tolerance

Tolerance is the level of error that is considered acceptable for some given application. A program has converged to a solution when it has found a solution with an error smaller than the tolerance.

Let $e=|f(x)|$ and $\text{tol}$ be the acceptable error. Then, since $f(x)=x^2+\frac{\text{tol}}{2}$ has no real roots, $|f(0)|=\frac{\text{tol}}{2}$ is an acceptable solution for a root finding program.

Notice that the tolerance and converging criteria must be considered carefully.

## 19.3 Bisection Method

The bisection method uses the intermediate value theorem to find roots. Let $f(x)$ be a continuous function and let $a,b\in\mathbb{R}$ such that $a< b$. Assume WLOG that $f(a)> 0$ and $f(b)< 0$. Then, by the intermediate value theorem, there exists a root on the interval $(a,b)$. Let $m=\frac{b+a}{2}$ be the midpoint between $a$ and $b$. If $f(m)=0$ or if it is close enough such that it is less than the tolerance, then $m$ is a root. If $f(m)$ is positive, then the root is contained in the interval $(m,b)$. If $f(m)$ is negative, then the root is contained in the interval $(a,m)$.

In [9]:
import numpy as np

def my_bisection(f, a, b, tol):
    # approximates a root, R, of f bounded
    # by a and b to within tolerance
    # | f(m) | < tol with m the midpoint
    # between a and b Recursive implementation

    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")

    # get midpoint
    m = (a + b)/2

    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a.
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b.
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

$\sqrt{2}$ can be computed as the root of the function $f(x)=x^2-2$. We know $f(0)< 0$ and $f(2)>2$, so they are acceptable bounds to start with.

In [10]:
f = lambda x: x**2 - 2

r1 = my_bisection(f, 0, 2, 0.1)
print("r1 =", r1)
r01 = my_bisection(f, 0, 2, 0.01)
print("r01 =", r01)

print("f(r1) =", f(r1))
print("f(r01) =", f(r01))

r1 = 1.4375
r01 = 1.4140625
f(r1) = 0.06640625
f(r01) = -0.00042724609375


In [11]:
my_bisection(f, 2, 4, 0.01)

Exception: The scalars a and b do not bound a root

## 19.4 Newton-Raphson Method
Let $f(x)$ be a smooth and continuous function and let $x_r$ be an unknown root of $f(x)$. Suppose that $x_0$ is a guess for $x_r$. If $x_0 \neq x_r$, then we would want to find a $x_1$ that is closer to the root. We can let $x_1$ be the intersection of the x-axis and the linear approximation of $f(x)$ around $x_0$. In other words,

\begin{equation*}
f(x) \approx f(x_0)+f'(x_0)(x-x_0) \implies x_1=x_0-\frac{f(x_0)}{f'(x_0)}.
\end{equation*}

A general Newton step computes an improved guess, $x_i$, using $x_{i-1}$ and is computed by $x_i=x_{i-1}-\frac{g(x_{i-1}}{g'(x_{i-1})}$.

If we again use $f(x)=x^2-2$ and let $x_0=1.4$ be the starting point, then we have:  

\begin{equation*}
x=1.4-\frac{1.4^2-2}{2(1.4}=1.4142857142857144.
\end{equation*}

In [16]:
import numpy as np

f = lambda x: x**2 - 2
f_prime = lambda x: 2*x
newton_raphson = 1.4 - (f(1.4))/(f_prime(1.4))

print("newton_raphson =", newton_raphson)
print("sqrt(2) =", np.sqrt(2))
print("error =", abs(np.sqrt(2) - newton_raphson))

newton_raphson = 1.4142857142857144
sqrt(2) = 1.4142135623730951
error = 7.215191261922271e-05


In [17]:
def my_newton(f, df, x0, tol):
    # output is an estimation of the root of f
    # using the Newton Raphson method
    # recursive implementation
    if abs(f(x0)) < tol:
        return x0
    else:
        return my_newton(f, df, x0 - f(x0)/df(x0), tol)

In [18]:
estimate = my_newton(f, f_prime, 1.5, 1e-6)
print("estimate =", estimate)
print("sqrt(2) =", np.sqrt(2))

estimate = 1.4142135623746899
sqrt(2) = 1.4142135623730951


##19.5 Root Finding in Python

Python has existing root-finding functions. One such example is *f_solve* from *scipy.optimize*. You can find the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html).

Suppose we want to find all roots of the function $f(x)=x^3-100x^2-x+100$ using *f_solve*.

In [19]:
from scipy.optimize import fsolve

In [27]:
f = lambda x: x**3-100*x**2-x+100

fsolve(f, [2, 80])

array([  1., 100.])